In [40]:
# you need to load these packages
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt
import libpysal

from pysal.model import spreg
from linearmodels.panel import PanelOLS
import esda  
%matplotlib inline


## The total structure is formulated as the follows

This file only contains the regressions analysis parts, for statistical analysis part please refer to the other notebook

1. Use Moran's I index to construct the result
2. Use Panel data to calculate the baseline results
3. Use Spatial Econometrics methods to construct the results
4. Heterogeneous tests and Robustness tests

In [41]:
data = pd.read_stata(r'regression_analysis.dta')

data['geometry'] = data['geometry'].apply(wkt.loads)
data = gpd.GeoDataFrame(data, geometry = data.geometry, crs = 'epsg:4326')
name_y = ['kou'] # dependent variable
name_x = ['end_price_pers', 'end_price_pers2', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park'] # independent variable
Y = data['kou'].values.reshape((-1, 1))
X_control = data[name_x].values

w = libpysal.weights.KNN.from_dataframe(data, k=5)
w.transform = 'r'

centroids = data.geometry.centroid # calculate the center of polygon
mi = esda.moran.Moran(Y, w)
print('The Moran\'s Index is: ', mi.I)
print('The p value is: ', mi.p_rand)

d:\anaconda\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 7998 disconnected components.
  warnings.warn(message)
C:\Users\Xuyuan\AppData\Local\Temp\ipykernel_19736\2046556725.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = data.geometry.centroid # calculate the center of polygon


The Moran's Index is:  0.7241685768466128
The p value is:  0.0


In [42]:

panel_data = pd.DataFrame(data)
panel_data = panel_data.set_index(['id_sp', 'year'])

model = PanelOLS.from_formula('kou ~ sanji + erji + yiji + end_price_pers + pop + light + pm25 + elect_store + kind + age + hotel_num + mall + museum_num + old + ktv + mid + primary + west_food + super + green_ratio + number_building + tihu + sub + floor_ratio + residence + park + TimeEffects', panel_data)
result = model.fit(cov_type="clustered", cluster_entity=True)
# you can also cluster at business area level
# .reformat_clusters(['bs_code'])


In [43]:
results1 = result.summary.tables[1]
re_result = pd.DataFrame(results1)
# print(result.summary)

## Panel FE Model

In [44]:

panel_data = pd.DataFrame(data)
panel_data = panel_data.set_index(['id_sp', 'year'])

model = PanelOLS.from_formula('kou ~ end_price_pers +  pop + light + pm25 + elect_store + kind + age + hotel_num + mall + museum_num + old + ktv + mid + primary + west_food + super + green_ratio + number_building + tihu + sub + floor_ratio + residence + park + TimeEffects + EntityEffects', panel_data)
result = model.fit(cov_type="clustered", cluster_entity=True)

results2 = result.summary.tables[1]
fe_result = pd.DataFrame(results2)

## Now Panel FE Lag Model

In [45]:
name_variables = ['kou', 'end_price_pers', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park']

df = data.drop_duplicates(subset = ['id_unique'], keep = 'first')
df.sort_values(by = ['id_unique', 'year'], inplace = True)
df.reset_index(drop = True, inplace = True)
w = libpysal.weights.KNN.from_dataframe(df, k=5)
w.transform = 'r'

C:\Users\Xuyuan\AppData\Local\Temp\ipykernel_19736\2499961660.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by = ['id_unique', 'year'], inplace = True)
d:\anaconda\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 35 disconnected components.
  warnings.warn(message)


In [46]:
year_list = [2016, 2017, 2018, 2019, 2020, 2021, 2022]

df_wide_merged = pd.DataFrame(df['id_unique'])
for i in name_variables:
    df_wide = data.pivot(index = 'id_unique', columns = 'year', values = i)
    for j in year_list:
        df_wide.rename(columns = {j: '{}{}'.format(i, j)}, inplace = True)
    df_wide.reset_index(drop = False, inplace = True)
    df_wide_merged = pd.merge(df_wide_merged, df_wide, on = 'id_unique')

df_wide_merged = pd.merge(df, df_wide_merged, on = 'id_unique')

In [47]:

name_y = ['kou']
name_x = ['end_price_pers', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park']

name_control = []
for i in name_x:
    for j in year_list:
        temp = str(i) + str(j)
        name_control.append(temp)

name_Y = []
for i in name_y:
    for j in year_list:
        temp = str(i) + str(j)
        name_Y.append(temp)

y = np.array([df_wide_merged[name] for name in name_Y]).T
X = np.array([df_wide_merged[name] for name in name_control]).T

In [48]:
print(y.shape)
print(X.shape)

(7998, 7)
(7998, 161)


In [49]:
# https://doi.org/10.1177/01600176032537
fe_lag = spreg.Panel_FE_Lag(y, X, w = w, name_y = name_y, name_x = name_x)

In [50]:
results3 = pd.DataFrame()
results3['betas'] = np.array(fe_lag.betas.flatten())
results3['name'] = fe_lag.name_x
results3['std'] = np.array(fe_lag.std_err)
results3['tval'] = np.array(fe_lag.z_stat)[:, 0]
results3['pval'] = np.array(fe_lag.z_stat)[:, 1]
# print(fe_lag.summary)

In [51]:
data = pd.read_stata(r'regression_analysis.dta')

data['geometry'] = data['geometry'].apply(wkt.loads)
data = gpd.GeoDataFrame(data, geometry = data.geometry, crs = 'epsg:4326')

lag_one_period_data = gpd.GeoDataFrame(data)
lag_one_period_data.set_index(['id_unique', 'year'], inplace = True)

name_y = ['kou']
name_x = ['end_price_pers', 'end_price_pers2', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park']

for i in name_x:
    lag_one_period_data[i] = lag_one_period_data[i].shift(1)

rows_to_drop = lag_one_period_data.loc[pd.IndexSlice[:, 2016], :].index

# Drop those rows.
lag_one_period_data = lag_one_period_data.drop(rows_to_drop)

In [52]:
lag_one_period_data.reset_index(drop = False, inplace = True)
Y = lag_one_period_data['kou'].values.reshape((-1, 1))
X_control = lag_one_period_data[name_x].values

w = libpysal.weights.KNN.from_dataframe(lag_one_period_data, k=5)
w.transform = 'r'

d:\anaconda\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 7998 disconnected components.
  warnings.warn(message)


In [53]:
name_variables = ['kou', 'end_price_pers', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park']

df = lag_one_period_data.drop_duplicates(subset = ['id_unique'], keep = 'first')
df.sort_values(by = ['id_unique', 'year'], inplace = True)
df.reset_index(drop = True, inplace = True)
w = libpysal.weights.KNN.from_dataframe(df, k=5)
w.transform = 'r'

year_list = [2017, 2018, 2019, 2020, 2021, 2022]

df_wide_merged = pd.DataFrame(df['id_unique'])
for i in name_variables:
    df_wide = lag_one_period_data.pivot(index = 'id_unique', columns = 'year', values = i)
    for j in year_list:
        df_wide.rename(columns = {j: '{}{}'.format(i, j)}, inplace = True)
    df_wide.reset_index(drop = False, inplace = True)
    df_wide_merged = pd.merge(df_wide_merged, df_wide, on = 'id_unique')

df_wide_merged = pd.merge(df, df_wide_merged, on = 'id_unique')



C:\Users\Xuyuan\AppData\Local\Temp\ipykernel_19736\2344216568.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by = ['id_unique', 'year'], inplace = True)
d:\anaconda\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 35 disconnected components.
  warnings.warn(message)


In [54]:

name_y = ['kou']
name_x = ['end_price_pers', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park']

name_control = []
for i in name_x:
    for j in year_list:
        temp = str(i) + str(j)
        name_control.append(temp)

name_Y = []
for i in name_y:
    for j in year_list:
        temp = str(i) + str(j)
        name_Y.append(temp)

y = np.array([df_wide_merged[name] for name in name_Y]).T
X = np.array([df_wide_merged[name] for name in name_control]).T



In [55]:
fe_lag = spreg.Panel_FE_Lag(y, X, w = w, name_y = name_y, name_x = name_x)

In [56]:
results4 = pd.DataFrame()
results4['betas'] = np.array(fe_lag.betas.flatten())
results4['name'] = fe_lag.name_x
results4['std'] = np.array(fe_lag.std_err)
results4['tval'] = np.array(fe_lag.z_stat)[:, 0]
results4['pval'] = np.array(fe_lag.z_stat)[:, 1]

In [57]:
result_list = [results1, results2, results3, results4]
names_of_table = ['re', 'fe', 'sl', 'sl_lag']
with pd.ExcelWriter(r'table 2.xlsx') as writer:
    for i in range(0, len(result_list)):
        result_list[i] = pd.DataFrame(result_list[i])
        result_list[i].to_excel(writer, sheet_name = names_of_table[i])
    writer.save()

C:\Users\Xuyuan\AppData\Local\Temp\ipykernel_19736\38827876.py:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
d:\anaconda\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [58]:
data = pd.read_stata(r'regression_analysis.dta')

data['geometry'] = data['geometry'].apply(wkt.loads)
data = gpd.GeoDataFrame(data, geometry = data.geometry, crs = 'epsg:4326')
name_y = ['kou']
name_x = ['end_price_pers', 'end_price_pers2', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park']
Y = data['kou'].values.reshape((-1, 1))
X_control = data[name_x].values

w = libpysal.weights.KNN.from_dataframe(data, k=5)
w.transform = 'r'


d:\anaconda\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 7998 disconnected components.
  warnings.warn(message)


In [59]:
condition = (data['city'] == 'tianjin') | (data['city'] == 'beijing')
jinjing = data[condition]
condition2 = (data['city'] == 'guangzhou') | (data['city'] == 'shenzhen')
zhusanjiao = data[condition2]
condition3 = (data['city'] == 'nanjing') | (data['city'] == 'hangzhou') | (data['city'] == 'shanghai')
husanjiao = data[condition3]
condition4 = (data['city'] == 'chengdu') | (data['city'] == 'chongqing')
chengyu = data[condition4]

resultx1 = pd.DataFrame()
resultx2 = pd.DataFrame()
resultx3 = pd.DataFrame()
resultx4 = pd.DataFrame()

result_list = [resultx1, resultx2, resultx3, resultx4]

year_list = [2016, 2017, 2018, 2019, 2020, 2021, 2022]

name_variables = ['kou', 'end_price_pers', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park']

name_space_cities = [jinjing, zhusanjiao, husanjiao, chengyu]

name_y = ['kou']
name_x = ['end_price_pers', 'pop', 'light', 'pm25', 'elect_store', 'kind', 'age', 'hotel_num', 'mall', 'museum_num', 'old', 'ktv', 'mid', 'primary', 'west_food', 'super', 'green_ratio', 'number_building', 'tihu', 'sub', 'floor_ratio', 'residence', 'park']

name_control = []
for i in name_x:
    for j in year_list:
        temp = str(i) + str(j)
        name_control.append(temp)

name_Y = []
for i in name_y:
    for j in year_list:
        temp = str(i) + str(j)
        name_Y.append(temp)

num_temp = 0
for i in name_space_cities:
    df = i.drop_duplicates(subset = ['id_unique'], keep = 'first')
    df.sort_values(by = ['id_unique', 'year'], inplace = True)
    df.reset_index(drop = True, inplace = True)

    w = libpysal.weights.KNN.from_dataframe(df, k=5)
    w.transform = 'r'

    df_wide_merged = pd.DataFrame(df['id_unique'])
    
    for z in name_variables:
        df_wide = i.pivot(index = 'id_unique', columns = 'year', values = z)
        for j in year_list:
            df_wide.rename(columns = {j: '{}{}'.format(z, j)}, inplace = True)
        df_wide.reset_index(drop = False, inplace = True)
        df_wide_merged = pd.merge(df_wide_merged, df_wide, on = 'id_unique')

    df_wide_merged = pd.merge(df, df_wide_merged, on = 'id_unique')
    # print(len(df_wide_merged.columns))
    # print(len(df_wide_merged))
    y = np.array([df_wide_merged[name] for name in name_Y]).T
    X = np.array([df_wide_merged[name] for name in name_control]).T
    
    
    fe_lag = spreg.Panel_FE_Lag(y, X, w = w, name_y = name_y, name_x = name_x)

    result_list[num_temp]['betas'] = np.array(fe_lag.betas.flatten())
    result_list[num_temp]['name'] = fe_lag.name_x
    result_list[num_temp]['std'] = np.array(fe_lag.std_err)
    result_list[num_temp]['tval'] = np.array(fe_lag.z_stat)[:, 0]
    result_list[num_temp]['pval'] = np.array(fe_lag.z_stat)[:, 1]
    
    num_temp += 1
    # print(fe_lag.summary)

C:\Users\Xuyuan\AppData\Local\Temp\ipykernel_19736\2054069920.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by = ['id_unique', 'year'], inplace = True)
d:\anaconda\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 6 disconnected components.
  warnings.warn(message)
C:\Users\Xuyuan\AppData\Local\Temp\ipykernel_19736\2054069920.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by = ['id_unique', 'year'], inplace = True)
d:\anaconda\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix 

In [60]:

with pd.ExcelWriter(r'table 3.xlsx') as writer:
    result_list[0].to_excel(writer, sheet_name = 'jinjing')
    result_list[1].to_excel(writer, sheet_name = 'zhusanjiao')
    result_list[2].to_excel(writer, sheet_name = 'changsanjiao')
    result_list[3].to_excel(writer, sheet_name = 'chengyu')
    writer.save()

C:\Users\Xuyuan\AppData\Local\Temp\ipykernel_19736\627678480.py:6: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
d:\anaconda\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
